In [ ]:
# from google.colab import files
# files.upload()
import os
# # Create the .kaggle directory
# os.makedirs('/root/.kaggle', exist_ok=True)
# # Move the kaggle.json file to the .kaggle directory
# !mv kaggle.json /root/.kaggle/
# !chmod 600 /root/.kaggle/kaggle.json

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("kamaruladha/mental-disorders-identification-reddit-nlp")

print("Path to dataset files:", path)

100%|██████████| 208M/208M [00:07<00:00, 28.3MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/kamaruladha/mental-disorders-identification-reddit-nlp/versions/2


In [ ]:
files = os.listdir(path)
print(files)

['mental_disorders_reddit.csv']


In [ ]:
data_path = path + '/' + files[0]
print(data_path)

/root/.cache/kagglehub/datasets/kamaruladha/mental-disorders-identification-reddit-nlp/versions/2/mental_disorders_reddit.csv


In [ ]:
import pandas as pd
df_complete = pd.read_csv(data_path)

# Assuming your dataset is a DataFrame called 'df'
df_complete = df_complete.drop(columns=['created_utc', 'over_18'])

# Display the updated DataFrame
df_complete.head()


,title,selftext,subreddit
0,Life is so pointless without others,Does anyone else think the most important part...,BPD
1,Cold rage?,Hello fellow friends 😄\n\nI'm on the BPD spect...,BPD
2,I don’t know who I am,My [F20] bf [M20] told me today (after I said ...,BPD
3,HELP! Opinions! Advice!,"Okay, I’m about to open up about many things I...",BPD
4,help,[removed],BPD


In [ ]:
# Remove rows where 'selftext' column has the value '[removed]'
df_complete = df_complete[df_complete['selftext'] != '[removed]']

# Display the updated DataFrame
df_complete.head(200)


,title,selftext,subreddit
0,Life is so pointless without others,Does anyone else think the most important part...,BPD
1,Cold rage?,Hello fellow friends 😄\n\nI'm on the BPD spect...,BPD
2,I don’t know who I am,My [F20] bf [M20] told me today (after I said ...,BPD
3,HELP! Opinions! Advice!,"Okay, I’m about to open up about many things I...",BPD
5,My ex got diagnosed with BPD,"Without going into detail, this diagnosis expl...",BPD
...,...,...,...
252,"""Oh, you mean the conservation we're not going...","No, you don't, you hated me and used me. You ...",BPD
253,"To all of you that suffer from BPD in Ukraine,...","I feel disgusted from what is happening, but c...",BPD
255,"2nd post, just joined",Normally I hate surprises but in this case thi...,BPD
257,Seeing disgusting stuff online from partners o...,So a lot of other subs I've read and groups ma...,BPD


In [ ]:
# Sample up to 1000 rows for each unique value in the 'subreddit' column
# More efficient sampling strategy
data = df_complete.groupby('subreddit').apply(
    lambda x: x.sample(n=min(1500, len(x)), random_state=42)
).reset_index(drop=True)

# Display the sampled DataFrame
data.head()


<ipython-input-7-4b46a279d299>:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data = df_complete.groupby('subreddit').apply(


,title,selftext,subreddit
0,I fucking hate my panic attacks and my tendenc...,"It has ruined so many experiences in my life, ...",Anxiety
1,I am all of a sudden terrified of a job I’ve b...,I remember for years before I ever applied her...,Anxiety
2,Woke up feeling strange,So yesterday I was outside in the sun all day ...,Anxiety
3,Worried for my mom,"Hello everyone, here I am writing because I wo...",Anxiety
4,My brain focuses on one person at a time to wo...,"I swear this is driving me crazy, it's like my...",Anxiety


In [ ]:
# Remove rows with NaN in the 'selftext' column
data = data.dropna(subset=['selftext'])

In [ ]:
# Combine 'title' and 'selftext' into a single column for analysis
data['text'] = data['title'].fillna('') + " " + data['selftext'].fillna('')

# Check for missing values
print(data.isnull().sum())

# Drop rows with missing target values (if any)
data = data.dropna(subset=['subreddit'])

# Encode the 'subreddit' column into numeric labels
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
data['label'] = label_encoder.fit_transform(data['subreddit'])

# Save the label classes for interpretation later
label_classes = label_encoder.classes_
print(f"Label classes: {label_classes}")

<ipython-input-9-124e24c2db2c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text'] = data['title'].fillna('') + " " + data['selftext'].fillna('')


title        0
selftext     0
subreddit    0
text         0
dtype: int64
Label classes: ['Anxiety' 'BPD' 'bipolar' 'depression' 'mentalillness' 'schizophrenia']


In [ ]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd

#dataset has already been loaded before

# Encode the 'subreddit' column
label_encoder = LabelEncoder()
data['encoded_labels'] = label_encoder.fit_transform(data['subreddit'])

# Save the label mappings
id2label = {idx: label for idx, label in enumerate(label_encoder.classes_)}
label2id = {label: idx for idx, label in id2label.items()}


In [ ]:
import numpy as np

# Save mappings
np.save("id2label.npy", id2label)
np.save("label2id.npy", label2id)


In [ ]:
label_classes = label_encoder.classes_
print(f"Label classes: {label_classes}")

Label classes: ['Anxiety' 'BPD' 'bipolar' 'depression' 'mentalillness' 'schizophrenia']


In [ ]:
# Split data into train and validation sets
from sklearn.model_selection import train_test_split

train_texts, val_texts, train_labels, val_labels = train_test_split(
    data['selftext'], data['encoded_labels'], test_size=0.2
)

# Ensure train_texts and val_texts contain only strings
train_texts = train_texts.astype(str) # Convert all elements to strings
val_texts = val_texts.astype(str)    # Convert all elements to strings

In [ ]:
# # Tokenize your text data
# from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
# import time
# import psutil
# import os
# import torch

# tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

# # Tokenization with efficient processing
# train_encodings = tokenizer(
#     list(train_texts),
#     truncation=True,
#     padding='max_length',
#     max_length=128,
#     return_tensors='pt'
# )
# val_encodings = tokenizer(
#     list(val_texts),
#     truncation=True,
#     padding='max_length',
#     max_length=128,
#     return_tensors='pt'
# )

# # Keep the same Dataset class
# class MentalHealthDataset(torch.utils.data.Dataset):
#     def __init__(self, encodings, labels):
#         self.encodings = encodings
#         self.labels = labels

#     def __len__(self):
#         return len(self.labels)

#     def __getitem__(self, idx):
#         item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
#         item['labels'] = torch.tensor(self.labels[idx])
#         return item

# train_dataset = MentalHealthDataset(train_encodings, train_labels.to_numpy())
# val_dataset = MentalHealthDataset(val_encodings, val_labels.to_numpy())

# # Use RoBERTa model instead of BERT
# model = RobertaForSequenceClassification.from_pretrained(
#     "roberta-base",
#     num_labels=len(id2label)
# )


# # Updated Training Arguments
# from transformers import TrainingArguments, Trainer, EarlyStoppingCallback

# training_args = TrainingArguments(
#     output_dir="./roberta_results",
#     evaluation_strategy="epoch",
#     save_strategy="epoch",
#     learning_rate=2e-5,
#     per_device_train_batch_size=16,  # Increased batch size
#     per_device_eval_batch_size=32,
#     num_train_epochs=10,  # More epochs with early stopping
#     warmup_steps=500,
#     weight_decay=0.01,
#     logging_dir="./roberta_logs",
#     logging_steps=10,
#     load_best_model_at_end=True,
#     metric_for_best_model='eval_loss'
# )

# # Trainer with Early Stopping Callback
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=val_dataset,
#     callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],  # Stop if no improvement
# )

# # Train the model
# trainer.train()
# metrics = trainer.evaluate()
# print("Validation metrics:", metrics)
# # Save the final model and tokenizer
# model.save_pretrained("./results")
# tokenizer.save_pretrained("./results")


In [ ]:
from transformers import (
    RobertaTokenizer,
    RobertaForSequenceClassification,
    Trainer,
    TrainingArguments,
    EarlyStoppingCallback
)
import torch
from torch.nn import functional as F
import numpy as np
from sklearn.model_selection import KFold
import random
import pandas as pd

def train_with_kfold(texts, labels, n_splits=2):
    # Convert inputs to numpy arrays if they're pandas Series/DataFrame
    if isinstance(texts, pd.Series):
        texts = texts.values
    if isinstance(labels, pd.Series):
        labels = labels.values

    # Ensure texts and labels are lists/arrays of the same length
    texts = list(texts)
    labels = np.array(labels)

    if len(texts) != len(labels):
        raise ValueError(f"Number of texts ({len(texts)}) doesn't match number of labels ({len(labels)})")

    # Print dataset statistics
    print(f"Dataset size: {len(texts)} samples")
    print(f"Number of unique labels: {len(np.unique(labels))}")

    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    fold_results = []

    for fold, (train_idx, val_idx) in enumerate(kf.split(texts)):
        print(f"\nTraining fold {fold + 1}/{n_splits}")
        print(f"Training size: {len(train_idx)}, Validation size: {len(val_idx)}")

        # Safely index the arrays
        train_texts = [texts[i] for i in train_idx]
        val_texts = [texts[i] for i in val_idx]
        train_labels = labels[train_idx]
        val_labels = labels[val_idx]

        # Verify data integrity
        print(f"Train texts: {len(train_texts)}, Train labels: {len(train_labels)}")
        print(f"Val texts: {len(val_texts)}, Val labels: {len(val_labels)}")

        # Create augmented dataset
        augmented_texts = []
        augmented_labels = []

        for text, label in zip(train_texts, train_labels):
            # Add original sample
            augmented_texts.append(str(text))  # Ensure text is string
            augmented_labels.append(int(label))  # Ensure label is integer

            # Add augmented version with 30% probability
            if random.random() < 0.3:
                aug_text = augment_text(str(text))
                augmented_texts.append(aug_text)
                augmented_labels.append(int(label))

        print(f"Augmented dataset size: {len(augmented_texts)}")

        # Tokenize
        tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

        try:
            train_encodings = tokenizer(
                augmented_texts,
                truncation=True,
                padding='max_length',
                max_length=128,
                return_tensors='pt'
            )

            val_encodings = tokenizer(
                val_texts,
                truncation=True,
                padding='max_length',
                max_length=128,
                return_tensors='pt'
            )
        except Exception as e:
            print(f"Tokenization error: {str(e)}")
            continue

        # Create datasets with proper type conversion
        train_dataset = MentalHealthDataset(
            train_encodings,
            np.array(augmented_labels, dtype=np.int64)
        )
        val_dataset = MentalHealthDataset(
            val_encodings,
            np.array(val_labels, dtype=np.int64)
        )

        # Initialize model
        model = RobertaForSequenceClassification.from_pretrained(
            "roberta-base",
            num_labels=len(np.unique(labels)),
            hidden_dropout_prob=0.2,
            attention_probs_dropout_prob=0.2
        )

        # Training arguments
        training_args = TrainingArguments(
            output_dir=f"./roberta_results/fold_{fold}",
            evaluation_strategy="steps",
            eval_steps=100,
            save_strategy="steps",
            save_steps=100,
            learning_rate=1e-5,
            per_device_train_batch_size=8,
            per_device_eval_batch_size=16,
            num_train_epochs=5,
            warmup_ratio=0.1,
            weight_decay=0.1,
            logging_dir=f"./roberta_logs/fold_{fold}",
            logging_steps=10,
            load_best_model_at_end=True,
            metric_for_best_model='eval_loss',
            max_grad_norm=1.0,
            label_smoothing_factor=0.1,
        )

        # Initialize trainer
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=val_dataset,
            callbacks=[
                EarlyStoppingCallback(
                    early_stopping_patience=3,
                    early_stopping_threshold=0.001
                )
            ],
        )

        try:
            # Train and evaluate
            trainer.train()
            metrics = trainer.evaluate()
            fold_results.append(metrics)

            # Save fold model
            model.save_pretrained(f"./results/fold_{fold}")

        except Exception as e:
            print(f"Training error in fold {fold + 1}: {str(e)}")
            continue

    return fold_results
# Augmentation function
def augment_text(text):
    """Simple augmentation using random word drops and shuffling"""
    words = text.split()
    if len(words) <= 3:
        return text

    # Randomly drop words (with probability)
    words = [w for w in words if random.random() > 0.1]

    # Randomly shuffle small segments
    if len(words) > 5:
        segment_length = random.randint(2, 3)
        for i in range(0, len(words) - segment_length, segment_length):
            if random.random() < 0.3:
                segment = words[i:i + segment_length]
                random.shuffle(segment)
                words[i:i + segment_length] = segment

    return ' '.join(words)

# Dataset class
class MentalHealthDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

# If your data is in pandas Series/DataFrame
fold_results = train_with_kfold(data['text'], data['label'])



Training fold 1/2
Training size: 3988, Validation size: 3988
Train texts: 3988, Train labels: 3988
Val texts: 3988, Val labels: 3988
Augmented dataset size: 5205

Epoch 5/5 [==============================================================] 651/651 10:13



,Step,Training Loss,Validation Loss
0,100,1.7642,1.787159
1,200,1.7713,1.764786
2,300,1.5527,1.482961
3,400,1.3900,1.311019
4,500,1.2366,1.268450
5,600,1.3944,1.244273


<ipython-input-21-f2c1e4665dfa>:189: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-21-f2c1e4665dfa>:189: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-21-f2c1e4665dfa>:189: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-21-f2c1e4665dfa>:189: UserWarning: To copy construct

,Step,Training Loss,Validation Loss
0,100,1.7878,1.782978
1,200,1.7273,1.702535
2,300,1.4753,1.358715
3,400,1.3641,1.247586
4,500,1.1995,1.206507


<ipython-input-21-f2c1e4665dfa>:189: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-21-f2c1e4665dfa>:189: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-21-f2c1e4665dfa>:189: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-21-f2c1e4665dfa>:189: UserWarning: To copy construct

In [ ]:
import numpy as np
from transformers import RobertaForSequenceClassification, RobertaTokenizer
import torch
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

def display_fold_results(fold_results):
    """Display averaged metrics across all folds"""
    print("\n=== K-Fold Cross Validation Results ===")

    # Extract metrics from each fold
    eval_losses = [fold['eval_loss'] for fold in fold_results]

    print(f"\nAverage Evaluation Loss: {np.mean(eval_losses):.4f} (±{np.std(eval_losses):.4f})")
    print("\nPer-fold Evaluation Loss:")
    for fold_idx, loss in enumerate(eval_losses):
        print(f"Fold {fold_idx + 1}: {loss:.4f}")

def evaluate_final_models(test_texts, test_labels, n_splits=2):
    """Evaluate all fold models and display combined metrics"""
    all_predictions = []

    # Load and evaluate each fold's model
    for fold in range(n_splits):
        model_path = f"./results/fold_{fold}"

        try:
            # Load model and tokenizer
            model = RobertaForSequenceClassification.from_pretrained(model_path)
            tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

            # Move model to GPU if available
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            model = model.to(device)
            model.eval()

            # Tokenize test data
            encodings = tokenizer(
                test_texts,
                truncation=True,
                padding='max_length',
                max_length=128,
                return_tensors='pt'
            )

            # Get predictions
            with torch.no_grad():
                input_ids = encodings['input_ids'].to(device)
                attention_mask = encodings['attention_mask'].to(device)
                outputs = model(input_ids=input_ids, attention_mask=attention_mask)
                predictions = torch.argmax(outputs.logits, dim=1).cpu().numpy()
                all_predictions.append(predictions)

        except Exception as e:
            print(f"Error evaluating fold {fold}: {str(e)}")
            continue

    # Ensemble predictions (majority voting)
    ensemble_predictions = np.stack(all_predictions)
    final_predictions = np.apply_along_axis(
        lambda x: np.bincount(x).argmax(),
        axis=0,
        arr=ensemble_predictions
    )

    # Calculate metrics
    accuracy = accuracy_score(test_labels, final_predictions)
    f1 = f1_score(test_labels, final_predictions, average='weighted')
    conf_matrix = confusion_matrix(test_labels, final_predictions)

    # Display results
    print("\n=== Ensemble Model Evaluation Results ===")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"F1 Score (weighted): {f1:.4f}")

    # Plot confusion matrix
    plt.figure(figsize=(10, 8))
    sns.heatmap(
        conf_matrix,
        annot=True,
        fmt='d',
        cmap='Blues'
    )
    plt.title('Ensemble Model Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.tight_layout()
    plt.savefig('./results/ensemble_confusion_matrix.png')
    plt.close()

    return {
        'accuracy': accuracy,
        'f1_score': f1,
        'confusion_matrix': conf_matrix
    }

# Usage example:

# First train the models
#fold_results = train_with_kfold(texts, labels, n_splits=2)

# Display fold results
display_fold_results(fold_results)

# Evaluate ensemble of all fold models
final_metrics = evaluate_final_models(
    test_texts=test_texts,
    test_labels=test_labels,
    n_splits=2
)



=== K-Fold Cross Validation Results ===

Average Evaluation Loss: 1.2217 (±0.0226)

Per-fold Evaluation Loss:
Fold 1: 1.1991
Fold 2: 1.2443

=== Ensemble Model Evaluation Results ===
Accuracy: 0.8460
F1 Score (weighted): 0.8480


In [ ]:
from transformers import RobertaForSequenceClassification, RobertaTokenizer
import torch
import numpy as np

def display_predictions(model_path, sentences, id2label):
    """
    Displays predictions for a given list of sentences.

    Args:
        model_path: Path to the saved model
        sentences: List of sentences to predict
        id2label: Dictionary mapping label ids to label names
    """
    # Load model and tokenizer
    model = RobertaForSequenceClassification.from_pretrained(
        model_path,
        num_labels=len(id2label)
    )
    tokenizer = RobertaTokenizer.from_pretrained(model_path)

    # Move model to GPU if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.eval()

    # Tokenize input sentences
    inputs = tokenizer(
        sentences,
        truncation=True,
        padding='max_length',
        max_length=128,
        return_tensors='pt'
    )

    # Move inputs to device
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    # Make predictions
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=1).cpu().numpy()

    # Display predictions
    for sentence, prediction in zip(sentences, predictions):
        label = id2label[prediction]
        print(f"Sentence: {sentence}")
        print(f"Predicted Label: {label}")
        print("-" * 50)

# Example sentences to predict
example_sentences = [
    "Today, I had a problem with my boss and a friend. We were nervous. My boss was furious about something but I defended myself and he became calm. Before that, he shouted.",
    "Life has been so stressful, I can't take it anymore.",
    "my therapist will ask me ab life goals and I just can’t imagine doing anything without someone by my side. but i hate them, i hate people",
    "do you ever have a sudden moment of clarity where you realize how dirty your room is, how your relationships are dying, how your life is falling apart, etc but the crushing weight of it all is so overwhelming that you just fall back into dissociation",
    "I think I'm struggling with something, but I don't know what."
]


# Replace {fold} with the appropriate fold index
model_path = "./roberta_results/fold_0"

# Display predictions
display_predictions(model_path, example_sentences, id2label)



Today, I had a problem with my boss and a friend. We were nervous. My boss was furious about something but I defended myself and he became calm. Before that, he shouted.
Possibly suffering from: anxiety

Life has been so stressful, I can't take it anymore.
Possibly suffering from: depression

my therapist will ask me ab life goals and I just can’t imagine doing anything without someone by my side. but i hate them, i hate people
Possibly suffering from: BPD

do you ever have a sudden moment of clarity where you realize how dirty your room is, how your relationships are dying, how your life is falling apart, etc but the crushing weight of it all is so overwhelming that you just fall back into dissociation
Possibly suffering from: schizophernia

I think I'm struggling with something, but I don't know what but it hurts.
Possibly suffering from: depression

